In [1]:
# Model Parameters

BATCH_SIZE = 32
GAUSSIAN_NOISE = 0.1
UPSAMPLING_MODE = 'Simple'

VALIDATION_SET_SIZE = 400
EPOCHS = 200

In [7]:
import os
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from skimage.io import imread
from skimage.morphology import label
from skimage.util import montage2d as montage
from skimage.segmentation import mark_boundaries

gc.enable()

In [9]:
montage_rgb = lambda x: np.stack([montage(x[:, :, :, i]) for i in range(x.shape[3])], -1)
input_dir = '../input'

train_image_dir = os.path.join(input_dir, '/train')
test_image_dir = os.path.join(input_dir, '/test')

In [ ]:
def rle_encode():
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


def rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    end = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.res


def multi_rle_encode(img):
    labels = label(img[:, :, 0])
    return [rle_encode(labels == k) for k in np.unique(label[labels>0])]